#  AccessGuruCorrect Notebook


This notebook demonstrates a full pipeline for: **AccessGuruCorrect**: Generate corrections using LLM prompting strategies

We’ll walk through each step with explanations and runnable code.

# AccessGuruCorrect LLM details
For AccessGuruCorrect, three LLMs were evaluated on syntax and layout accessibility violations:
* GPT-4-0125-preview,
* Mistral-7B-v0.1, and
* Qwen2.5-Coder

And for correction on semantic accessibility violations requiring multimodal reasoning capabilities to reason over images, the LLMs used are:
* GPT-4,
* Pixtral-12B, and
* Qwen-VL.

If you want to mimic the paper, please use the above mentioned models.

**Note**: In this notebook, we will use Mistral for Syntax and Layout Correction while Qwen for Semantic Correction for these are the free models which we can test.
* Mistral: https://openrouter.ai/mistralai/mistral-7b-instruct:free
* Qwen: https://openrouter.ai/qwen/qwen2.5-vl-72b-instruct:free



## 2.1. API key setup from [OpenRouter](https://openrouter.ai/)

### API Access Guide
1. Open https://openrouter.ai/settings/keys
2. Click on "Create API key"
4. Create the api and copy the key to the clipboard

In [ ]:
# Paste the api key here:
API_KEY = "sk-or-v1-xxxxxxx"  # Replace with your actual API key
Openrouter_API_URL = "https://openrouter.ai/api/v1/chat/completions"

mistral_model = "mistralai/mistral-7b-instruct:free"
qwen_model = "qwen/qwen2.5-vl-72b-instruct:free"


## 2.2. Imports & Setups


In [ ]:
import os
import json
import pandas as pd
import requests
import re
import time
import csv
from openai import OpenAI
import base64

In [ ]:
# Output directories
output_dir = "/content/html_pages_async"
screenshot_dir = "/content/element_screenshots"
os.makedirs(output_dir, exist_ok=True)
os.makedirs(screenshot_dir, exist_ok=True)

In [ ]:
# Download required data from AccessGuru Repo
! wget 'https://raw.githubusercontent.com/NadeenAhmad/AccessGuruLLM/refs/heads/main/data/prompts_support/WCAGGuidelines.csv'

--2025-08-23 01:44:55--  https://raw.githubusercontent.com/NadeenAhmad/AccessGuruLLM/refs/heads/main/data/prompts_support/WCAGGuidelines.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20422 (20K) [text/plain]
Saving to: ‘WCAGGuidelines.csv’

WCAGGuidelines.csv  100%[===================>]  19.94K  --.-KB/s    in 0.003s  

2025-08-23 01:44:55 (6.02 MB/s) - ‘WCAGGuidelines.csv’ saved [20422/20422]



In [ ]:
! wget 'https://raw.githubusercontent.com/NadeenAhmad/AccessGuruLLM/refs/heads/main/data/prompts_support/violation_taxonomy.csv'
! wget 'https://raw.githubusercontent.com/NadeenAhmad/AccessGuruLLM/refs/heads/main/data/prompts_support/mapping_dict_file.json'


--2025-08-23 01:44:56--  https://raw.githubusercontent.com/NadeenAhmad/AccessGuruLLM/refs/heads/main/data/prompts_support/violation_taxonomy.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2572 (2.5K) [text/plain]
Saving to: ‘violation_taxonomy.csv’

violation_taxonomy. 100%[===================>]   2.51K  --.-KB/s    in 0s      

2025-08-23 01:44:56 (45.6 MB/s) - ‘violation_taxonomy.csv’ saved [2572/2572]

--2025-08-23 01:44:56--  https://raw.githubusercontent.com/NadeenAhmad/AccessGuruLLM/refs/heads/main/data/prompts_support/mapping_dict_file.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:44

## 2.3. Utility Functions

Functions to get the guidelines of the Violation and the violation category of descriptions


In [ ]:
def get_guidelines(violationType):
  guide = []
  json_file_path = '/content/mapping_dict_file.json'
  # Open and read the JSON file
  with open(json_file_path, 'r') as file:
     data = json.load(file)
  guidelineNames=data[violationType]
  guidelines= pd.read_csv('/content/WCAGGuidelines.csv')
  for x in guidelineNames:
      guide.append(str(x+":" +guidelines[guidelines['guideId'] == x]['description'].values[0]))
  return guide


# get_guidelines("image-alt-not-descriptive")

def get_category(c):
    categories = {
        "Syntax": """These occur when HTML code lacks essential structural elements or attributes required for accessibility,
        such as missing alt attributes or improper use of heading tags and table elements.
        To correct this type of web accessibility violation, expertise in HTML coding is required.""",
        "Layout": """These relate to design and visual organization issues, such as insufficient color contrast or poor font sizing,
        that reduce readability or usability for visually impaired users.
        Fixes must ensure accessibility without negatively affecting users without impairments or distort the page layout for users without impairments.
        To correct this type of web accessibility violation, knowledge of web design principles and front-end coding is required.""",
        "Semantic": """These involve the misuse or absence of meaningful content or attributes,
        such as vague alt text or improper use of semantic elements like <header> or <section>.
    To correct this type of web accessibility violation, expertise in HTML semantics and coding is required.""",
    }
    return categories.get(c, "No description available for this category.")


## 2.4. LLM Prompting: Metacognitive Prompting
Metacognitive Prompting, metacognition refers to the
ability to self-reflect and critically evaluate one’s own cognitive
processes [1, 2]. Metacognitive prompting [3, 4] mimics
human cognitive steps through the following stages:
* (a) Selfunderstanding: the model assesses its understanding of the
prompt by identifying and interpreting relevant information,
* (b) Reflection: the model undergoes preliminary judgment,
forming an initial response, followed by inference evaluation,
where it reflects on and refines its initial interpretation, and
* (c) Self-regulation: the model finalizes its response and performs a confidence assessment to evaluate the reliability of its
decision.

Ref:
- [1] Stephen M Fleming and Hakwan C Lau. 2014. How to measure metacognition. Frontiers in human neuroscience 8 (2014), 443.
- [2] Chris D Frith. 2012. The role of metacognition in human social interactions.
Philosophical Transactions of the Royal Society B: Biological Sciences 367,
1599 (2012), 2213–2223.
- [3] Yuqing Wang and Yun Zhao. 2023. Metacognitive Prompting Improves
Understanding in Large Language Models. CoRR abs/2308.05342 (2023).
doi:10.48550/ARXIV.2308.05342 arXiv:2308.05342
- [4] Yujia Zhou, Zheng Liu, Jiajie Jin, Jian-Yun Nie, and Zhicheng Dou. 2024.
Metacognitive Retrieval-Augmented Large Language Models. In Proceedings of the ACM on Web Conference 2024, WWW 2024, Singapore, May 13-17,
2024, Tat-Seng Chua, Chong-Wah Ngo, Ravi Kumar, Hady W. Lauw, and
Roy Ka-Wei Lee (Eds.). ACM, 1453–1463. doi:10.1145/3589334.3645481

In [ ]:
def send_prompt(API_KEY,prompt, chat_history, img_path=None):
    """Send a structured prompt and optional image to an LLM while maintaining conversation history."""

    time.sleep(10)  # Avoid flooding the API

    # Step 1: Prepare content
    if img_path:
        with open(img_path, "rb") as image_file:
            encoded_image = base64.b64encode(image_file.read()).decode("utf-8")
            image_url = f"data:image/png;base64,{encoded_image}"
        user_content = [
            {"type": "text", "text": prompt},
            {"type": "image_url", "image_url": {"url": image_url}}
        ]
    else:
        user_content = [{"type": "text", "text": prompt}]

    # Step 2: Append prompt to history
    chat_history.append({"role": "user", "content": user_content})

    # Step 3: Call the API
    response = requests.post(
        Openrouter_API_URL,
        headers={
            "Authorization": f"Bearer {API_KEY}",
            "Content-Type": "application/json"
        },
        json={
            "model": model_name,
            "messages": chat_history
        }
    )

    # Step 4: Process response
    if response.status_code == 200:
        chat_response = response.json()["choices"][0]["message"]["content"]
        chat_history.append({"role": "assistant", "content": chat_response})
        return chat_response
    else:
        return f"Error: {response.status_code}, {response.text}"

In [ ]:
# Function to perform metacognitive prompting step-by-step
def analyze_web_accessibility_violation(API_KEY,category, category_description, violationType, violationDescription, impact, URL, HTMLElement, guideline,img_path):
    """Executes the metacognitive evaluation workflow for web accessibility violations."""
    # Initialize chat history
    chat_history = [
        {"role": "system", "content": """You are a Web accessibility expert with a strong proficiency in HTML and a deep commitment to fixing Web accessibility violations.
                                        You are familiar with different types of disabilities and user needs.
                                        You are familiar with assistive technologies such as screen readers and understand how they interact with the Web.
                                        You specialize in analyzing Web pages, identifying issues, and providing immediate, corrected HTML code solutions that meet WCAG 2.1 standards.
                                        Your expertise includes resolving problems like missing or improper Alt text, insufficient heading structure, non-semantic elements, inaccessible forms, and color contrast issues.
                                        You are adept at transforming flawed code into compliant, clean HTML that works seamlessly with assistive technologies, ensuring that Websites are fully navigable by keyboard and readable by screen readers.
                                        You provide the corrected code necessary for immediate implementation, ensuring that Websites are not only compliant but truly inclusive for users with disabilities.
                                        Your mission is to ensure that every Website and Web application is accessible to all users by providing expertly corrected HTML, making the Web a more inclusive space.
                                        You reflect on your own answers, assess their accuracy, and provide corrections only when necessary."""}
    ]
    # Step 1: Comprehension & Clarification
    if category == "Semantic":
        prompt_comprehension_clarification = f"""
            You are analyzing a web accessibility issue.
            You are a multimodal accessibility auditor. Before reasoning about the issue, first interpret the attached screenshot image, which shows the specific part of a web page that may violate accessibility guidelines. You are also given supporting metadata and the corresponding HTML.

            Begin your analysis by visually examining the image. Then proceed to correlate it with the HTML and metadata to validate the violation and propose a fix.

            ### Your tasks:
            1. **Interpret the visual content** of the attached image.
            2. **Identify the UI element** (e.g. a button or icon) shown in the image.
            3. **Determine whether the element is accessible** (e.g., does it have a clear label, alt text, and function?).
            4. **Compare your findings** with the **corresponding HTML** provided and highlight any mismatches.
            5. **Suggest an accessibility-compliant fix** if there's a violation.

            ### Metadata:
            Violation Category: "{category}": {category_description},
            Violation type: "{violationType}",
            Violation description: "{violationDescription}",
            Impact: "{impact}" (Impact is a rating determined by the severity of the violation, indicating the extent to which it hinders user interaction with the Web content. The scale is [cosmetic, minor, moderate, serious, critical]).
            Web page URL: "{URL}"
            Affected HTML Element(s): "{HTMLElement}"
        """
        print("\n### Step 1: Comprehension Clarification ###")
        s1 = send_prompt(API_KEY,prompt_comprehension_clarification, chat_history,img_path)
        # print(s1)
    else:
        # for Syntax and Layout
        prompt_comprehension_clarification = f"""
            Clarify your understanding of the following web accessibility violation:
            Violation Category: "{category}": {category_description},
            Violation type: "{violationType}",
            Violation description: "{violationDescription}",
            Impact: "{impact}" (Impact is a rating determined by the severity of the violation, indicating the extent to which it hinders user interaction with the Web content. The scale is [cosmetic, minor, moderate, serious, critical]).
            Web page URL: "{URL}"
            Affected HTML Element(s): "{HTMLElement}"
        """
        print("\n### Step 1: Comprehension Clarification ###")
        s1 = send_prompt(API_KEY,prompt_comprehension_clarification, chat_history)
        # print(s1)

    # Step 2: Preliminary Judgment & Correction
    prompt_preliminary_judgment = f"""
    Based on your understanding, provide a preliminary correction for the web accessibility violation based on the following WCAG guideline(s): "{guideline}".
    Make sure your generated code corrects the web accessibility violation without introducing new violations.
    Ensure you generate the complete corrected code, not just a snippet.
    """
    print("\n### Step 2: Preliminary Correction ###")
    s2 = send_prompt(API_KEY,prompt_preliminary_judgment, chat_history)
    # print(s2)

    # Step 3: Critical Evaluation of the Correction
    prompt_critical_evaluation = """
    Critically assess your preliminary correction, make sure to correct the initial web accessibility violation without introducing new web accessibility violations.
    Only make corrections if the previous answer is incorrect.
    Make sure your generated code corrects the web accessibility violation without introducing new violations.
    """
    print("\n### Step 3: Critical Evaluation ###")
    s3 = send_prompt(API_KEY,prompt_critical_evaluation, chat_history)
    # print(s3)

    # Step 4: Decision Confirmation
    prompt_decision_confirmation = """
    Confirm your final decision on whether the correction is accurate or not and provide the reasoning for your decision.
    Only suggest further corrections if the initial response contains errors.
    Make sure your generated code corrects the web accessibility violation without introducing new violations.
    Enclose your corrected HTML code to replace the initial code with violations between these two marker strings: "###albidaya###" as first line and "###alnihaya###" as last line.
    """
    print("\n### Step 4: Decision Confirmation ###")
    s4 = send_prompt(API_KEY,prompt_decision_confirmation, chat_history)

    # Step 5: Confidence Level Evaluation
    # prompt_confidence_level = """
    # Evaluate your confidence (0-100%) in your correction, enclose your confidence score between these two marker strings: "###albidaya###" as first line and "###alnihaya###" as last line.
    # Provide an explanation for this confidence level, enclose your explanation between these two marker strings: "###albidaya2###" as first line and "###alnihaya2###" as last line.
    # """
    # s6 = send_prompt(prompt_confidence_level, chat_history)

    prompt_confidence_level = """
    Evaluate your confidence (0-100%) in your correction, enclose your confidence score as "Score: <score>".
    Provide an explanation for this confidence level, enclose your explanation as "Explanation: <explanation>".
    """
    print("\n### Step 5: Confidence Level Evaluation ###")
    s5 = send_prompt(API_KEY,prompt_confidence_level, chat_history)

    # Pattern to extract between markers
    # match1 = re.search(r'###albidaya###(.*?)###alnihaya###', s5, re.DOTALL)
    # match2 = re.search(r'###albidaya2###(.*?)###alnihaya2###', s5, re.DOTALL)
    pattern = r"Score:\s*(\d+)%?\s*Explanation:\s*(.*)"
    match = re.search(pattern, s5, re.DOTALL)

    if match:
        try:
            Confidence_Score = match.group(1)
        except:
            Confidence_Score = ""
        try:
            Explanation = match.group(2)
        except:
            Explanation = ""


    else:
      Confidence_Score = ""
      Explanation = ""

    return s4,Confidence_Score,Explanation,s5

## 2.5. AccessCorrect

## 2.5.a AccessCorrect on the AccessDetect Dataset Created
Running on the dataset created from 1. [AcessGuruDetect](https://colab.research.google.com/drive/1edKtrSCJ2FrZqU0G8v9424yKG-qQrrRU?usp=sharing) and saved in 1.7. section. Skip to section 2.6. if you want to test a single instance.

**Note**: Skip if you didn't run AccessGuru Detec Notebook

In [ ]:
violation_df = pd.read_csv("/content/DetectedViolationData.csv")

violation_df.head(2)

In [ ]:
violation_df.columns

## 2.5.a.1. Setups for the final csv

In [ ]:
output_path = "/content"
output_filename = "CorrectedViolationData.csv"
filename = os.path.join(output_path, output_filename)
file_exists = os.path.exists(filename)

new_column1 = "Decision_Confirmation"
new_column2 = "Confidence_Score"
new_column3 = "Explanation"
new_column4 = "s5_OrgPrompt"

data_fields = ['id', 'web_URL_id', 'web_URL', 'scrape_status', 'violation_count',
       'violation_name', 'violation_score', 'violation_description',
       'violation_description_url', 'domain_category', 'violation_impact',
       'wcag_reference', 'violation_category', 'html_file_name',
       'affected_html_elements', 'supplementary_information',new_column1,new_column2,new_column3,new_column4]

## 2.5.a.2 Run on the dataset


In [ ]:
model_name = mistral_model

with open(filename, "a") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=data_fields)

    if not file_exists:
        writer.writeheader()
    for index, row in violation_df.iterrows():
        print("==> Processing index", index)

        violation_category = row['violation_category']
        category_description = get_category(violation_category)
        violationType = row["violation_name"]

        Decision_Confirmation, Confidence_Score, Explanation,s5 = analyze_web_accessibility_violation(API_KEY,
                                                                      category = violation_category,
                                                                      category_description = category_description,
                                                                      violationType=row["violation_name"],
                                                                      violationDescription=row['violation_description'],
                                                                      impact=row['violation_impact'],
                                                                      URL=row['web_URL'],
                                                                      HTMLElement=row['affected_html_elements'],
                                                                      guideline = get_guidelines(violationType),
                                                                      img_path=None
                                                                  )

        mydict = {
            'id':row['id'],
            'web_URL_id': row['web_URL_id'],
            'web_URL':row['web_URL'],
            'scrape_status':row['scrape_status'],
            'violation_count':row["violation_count"],
            'violation_name':row["violation_name"],
            'violation_score':row['violation_score'],
            'violation_description':row['violation_description'],
            'violation_description_url':row['violation_description_url'],
            'domain_category':row['domain_category'],
            'affected_html_elements':row['affected_html_elements'],
            'html_file_name':row['html_file_name'],
            'supplementary_information':row['supplementary_information'],
            'violation_impact':row['violation_impact'],
            'wcag_reference':row['wcag_reference'],
            'violation_category':row['violation_category'],
             new_column1: Decision_Confirmation,
            new_column2: Confidence_Score,
            new_column3:Explanation,
            new_column4:s5
        }

        writer.writerows([mydict])
    print(f"Experiment completed! Results saved to {filename}")

==> Processing index 0

### Step 1: Comprehension Clarification ###

### Step 2: Preliminary Correction ###

### Step 3: Critical Evaluation ###

### Step 4: Decision Confirmation ###

### Step 5: Confidence Level Evaluation ###
==> Processing index 1

### Step 1: Comprehension Clarification ###

### Step 2: Preliminary Correction ###

### Step 3: Critical Evaluation ###

### Step 4: Decision Confirmation ###

### Step 5: Confidence Level Evaluation ###
==> Processing index 2

### Step 1: Comprehension Clarification ###

### Step 2: Preliminary Correction ###

### Step 3: Critical Evaluation ###

### Step 4: Decision Confirmation ###

### Step 5: Confidence Level Evaluation ###
Experiment completed! Results saved to /content/CorrectedViolationData.csv


In [ ]:
corrected_df = pd.read_csv('/content/CorrectedViolationData.csv')
corrected_df.head(2)

,id,web_URL_id,web_URL,scrape_status,violation_count,violation_name,violation_score,violation_description,violation_description_url,domain_category,violation_impact,wcag_reference,violation_category,html_file_name,affected_html_elements,supplementary_information,Decision_Confirmation,Confidence_Score,Explanation,s5_OrgPrompt
0,1_0,1,https://www.ki.uni-stuttgart.de/,scraped,3,aria-allowed-attr,4,Ensures ARIA attributes are allowed for an ele...,https://dequeuniversity.com/rules/axe/4.4/aria...,Educational Platforms,serious,"['4.1.2 Name, Role, Value']",Syntax,/content/html_pages_async/www_ki_uni-stuttgart...,"<span class=""box__item-icon is-profile"" data-t...",NaN,"Error: 429, {""error"":{""message"":""Rate limit ex...",NaN,NaN,"Error: 429, {""error"":{""message"":""Rate limit ex..."
1,1_1,1,https://www.ki.uni-stuttgart.de/,scraped,3,heading-order,3,Ensures the order of headings is semantically ...,https://dequeuniversity.com/rules/axe/4.4/head...,Educational Platforms,moderate,['1.3.1 Info and Relationships'],Syntax,/content/html_pages_async/www_ki_uni-stuttgart...,"<h3 class=""panel-title"">Our Departments</h3>",NaN,"Error: 429, {""error"":{""message"":""Rate limit ex...",NaN,NaN,"Error: 429, {""error"":{""message"":""Rate limit ex..."


## 2.5.b For Semantic Violation Dataset:
If you have your Semantic violation Dataset, <br>
PLEASE MAKE SURE THAT THE SCREENSHOTS ARE SAVED WITH THE ID NAME.

**Example**: if id is 1_0, screenshot image name should be 1_0.png

**Note**: element_screenshots folder is for the supplementary information images.

Or you can download [AccessGuru Sampled Semantic](https://raw.githubusercontent.com/NadeenAhmad/AccessGuruLLM/refs/heads/main/data/accessguru_dataset/accessguru_sampled_semantic_violations.csv) dataset along with the [Screenshots](https://github.com/NadeenAhmad/AccessGuruLLM/tree/main/data/accessguru_dataset/accessguru_semantic_violations_sampled_dataset_supp_material) to test

Or run the sample test below in Section 2.6.a



---

# 2.6. Sample Test Runs

## 2.6.a. Example Run: Semantic

In [ ]:
# download sample image from AccessGuru Repo
! wget "https://raw.githubusercontent.com/NadeenAhmad/AccessGuruLLM/main/data/accessguru_dataset/accessguru_semantic_violations_sampled_dataset_supp_material/1.png"

--2025-08-23 01:45:55--  https://raw.githubusercontent.com/NadeenAhmad/AccessGuruLLM/main/data/accessguru_dataset/accessguru_semantic_violations_sampled_dataset_supp_material/1.png
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25543 (25K) [image/png]
Saving to: ‘1.png’

1.png               100%[===================>]  24.94K  --.-KB/s    in 0.001s  

2025-08-23 01:45:55 (18.5 MB/s) - ‘1.png’ saved [25543/25543]



In [ ]:
# Use the free model to test:
# Use Mistral for Syntax and Layout Correction
# Use Qwen for Semantic Correction

model_name = qwen_model

violationType = "image-alt-not-descriptive"
HTMLElement = """
                  <html lang="en">    <img alt="ERCIM logo" src="image.png"/>    </html>
              """
url = "https://www.w3.org/WAI/content-assets/wcag-act-rules/testcases/qt1vmo/485f10faf222cd48fea2ab3ee79c2d354e51ea33.html"
violationDescription = "Inaccurate or misleading alternative text that fails to describe the purpose of the image."
img_path = "/content/1.png"

s4, Confidence_Score, Explanation, s5 = analyze_web_accessibility_violation(API_KEY,
    category = "Semantic",
    category_description = get_category("Semantic"),
    violationType=violationType,
    violationDescription=violationDescription,
    impact="critical",
    URL=url,
    HTMLElement=HTMLElement,
    guideline = get_guidelines(violationType),
    img_path=img_path
)

print("====Decision_Confirmation====")
print(s4)
print()
print("====Confidence_Score====")
print(Confidence_Score)
print()
print("====Explanation====")
print(Explanation)
print()
print("====s5_OrgPrompt====")
print(s5)
print()


### Step 1: Comprehension Clarification ###

### Step 2: Preliminary Correction ###

### Step 3: Critical Evaluation ###

### Step 4: Decision Confirmation ###

### Step 5: Confidence Level Evaluation ###
====Decision_Confirmation====
###albidaya###
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>World Wide Web Consortium (W3C) Logo</title>
</head>
<body>
    <header>
        <img alt="W3C logo" src="image.png"/>
    </header>
    <main>
        <!-- Add main content here -->
    </main>
    <footer>
        <!-- Add footer content here -->
    </footer>
</body>
</html>
###alnihaya###

### Confirmation and Reasoning

#### Confirmation
The final correction is accurate and addresses the initial web accessibility violation without introducing new violations. The `alt` attribute has been updated to accurately describe the W3C logo, ensuring that the image is accessible to users relying

## 2.6.b Example Run: Layout

In [ ]:
# Use the free model to test:
# Use Mistral for Syntax and Layout Correction
# Use Qwen for Semantic Correction

model_name = mistral_model

violationType = "color-contrast-enhanced"
HTMLElement = """
                 <a href="/buy-from-government">How to buy from the U.S. government</a>, <a href="/holidays">Learn about American holidays</a>, <a href="/agency-index">Find government agencies</a>
             """
url = "https://www.usa.gov/about-the-us"
violationDescription="Ensures the contrast between foreground and background colors meets WCAG 2 AAA contrast ratio thresholds."

s4,Confidence_Score,Explanation,s5 = analyze_web_accessibility_violation(API_KEY,
    category = "Layout",
    category_description = get_category("Layout"),
    violationType = violationType,
    violationDescription=violationDescription,
    impact = "serious",
    URL = url,
    HTMLElement = HTMLElement,
    guideline = get_guidelines(violationType),
    img_path = None
)

print("====Decision_Confirmation====")
print(s4)
print()
print("====Confidence_Score====")
print(Confidence_Score)
print()
print("====Explanation====")
print(Explanation)
print()
print("====s5_OrgPrompt====")
print(s5)
print()


### Step 1: Comprehension Clarification ###

### Step 2: Preliminary Correction ###

### Step 3: Critical Evaluation ###

### Step 4: Decision Confirmation ###

### Step 5: Confidence Level Evaluation ###
====Decision_Confirmation====
 My final decision on the correction is that it is accurate based on the given information and constraints. I took the decision after adjusting the media queries to provide a more inclusive set of adjustable text sizes for users with a range of screen resolutions, ensuring accessibility for visually impaired users without introducing new web accessibility violations. Here's the corrected HTML code enclosed between the given marker strings "###albidaya###" and "###alnihaya###":

```html
###albidaya###
<style>
    @media screen and (min-width: 37.5em) {
            a {
                font-size: calc(14px + (3 * (100vw - 360px) / 1680) * 2);
            }
        }
        @media screen and (min-width: 45em) {
            a {
                font-size: cal

## 2.5.c. Example Run: Syntax

In [ ]:
# Use the free model to test:
# Use Mistral for Syntax and Layout Correction
# Use Qwen for Semantic Correction

model_name = mistral_model

violationType = "duplicate-id-active"
HTMLElement = """
                <button type="button" id="fides-accept-all-button" data-testid="I Accept-btn" class="fides-banner-button fides-banner-button-primary fides-accept-all-button" style="cursor: pointer;">I Accept</button>, <button type="button" role="menuitemradio" aria-checked="true" data-testid="fides-i18n-option-en" id="fidesActiveMenuItem" title="English" aria-label="English" tabindex="-1" class="fides-banner-button fides-menu-item ">English</button>
              """
url = "https://arstechnica.com/ai/"
violationDescription = "Ensures every id attribute value of active elements is unique."

s4, Confidence_Score, Explanation, s5 = analyze_web_accessibility_violation(API_KEY,
    category = "Syntax",
    category_description = get_category("Syntax"),
    violationType=violationType,
    violationDescription=violationDescription,
    impact="serious",
    URL=url,
    HTMLElement=HTMLElement,
    guideline = get_guidelines(violationType),
    img_path=None
)

print("====Decision_Confirmation====")
print(s4)
print()
print("====Confidence_Score====")
print(Confidence_Score)
print()
print("====Explanation====")
print(Explanation)
print()
print("====s5_OrgPrompt====")
print(s5)
print()


### Step 1: Comprehension Clarification ###

### Step 2: Preliminary Correction ###

### Step 3: Critical Evaluation ###

### Step 4: Decision Confirmation ###

### Step 5: Confidence Level Evaluation ###
====Decision_Confirmation====
 My initial correction addresses the duplicate ID violation in the provided HTML code by assigning separate unique IDs to both buttons. The corrected code does not introduce any new accessibility violations.

Below is the corrected HTML code, enclosed between the marker strings:

```html
###albidaya###
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Ars Technica - AI</title>
</head>
<body>
    <header role="banner">
        <h1>Welcome to Ars Technica AI</h1>
    </header>

    <nav role="navigation">
        <ul>
            <li><a href="#">Home</a></li>
            <li><a href="#">About</a></li>
            <!-- Add more navigation items as needed -